In [1]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [3]:
data=pd.read_csv('../hindi.csv')

In [4]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())



In [5]:
with open('pickles/posting_list.pkl','rb') as file:
    tf=pickle.load(file)
    file.close()
    
with open('pickles/df.pkl','rb') as file:
    DF=pickle.load(file)
    file.close()
    
with open('pickles/file_idx.pkl','rb') as file:
    file_idx=pickle.load(file)
    file.close()
    
with open('pickles/doc_len.pkl','rb') as file:
    doc_len=pickle.load(file)
    file.close()

In [6]:
k=0
Ld=doc_len
N=len(file_idx)
for i in Ld:
    k+=Ld[i]
Lavg=k/N

In [7]:
def IDF(q):
    DF1=0
    if q in DF:
        DF1=DF[q]
    ans=math.log((N-DF1+0.5)/(DF1+0.5))
    return ans
k=1.2
b=0.75

In [8]:
def bm25(query,counter=10):
    score={}
    for i in range(len(file_idx)):
        score[i]=0
    text = remove_special_characters(query)
    words = text.split()
    words=[word for word in words if word not in stopwords]
    words=[word for word in words if word in DF.keys()]
    for i in range(len(file_idx)):
        score[i]=0
        for qi in words:
            TF=0
            if qi in tf:
                if i in tf[qi]:
                    TF=tf[qi][i]
            idf=IDF(qi)
            ans=idf*(k+1)*TF/(TF+k*(1-b+b*(Ld[i]/Lavg)))
            score[i]+=ans
    score=sorted(score.items(),key=lambda item: item[1],reverse=True)
    verse_index=[]
    for i in score:
        if counter== 0:
            break
        verse_index.append(i[0])
        counter-=1
    return verse_index

In [9]:
verse_index=bm25('योग्यता के बिना किसी भी लक्ष्य की प्रप्ति नहीं होती। इस श्लोक में वर्णित आवश्यक गुणों को समझने से यह भी स्पष्ट हो जायेगा कि अनेक वर्षों तक साधना करने पर भी कुछ साधक लक्ष्य तक क्यों नहीं पहुँच पाते हैं। श्रद्धा तत्परता (भक्ति) तथा इंद्रिय संयम  ये वे तीन अत्यावश्यक गुण हैं जिनके द्वारा जीवत्व के बन्धनों से मुक्त होकर हम देवत्व को प्राप्त करने की आशा कर सकते हैं। परन्तु इन तीनों शब्दों के अर्थों के विषय में अनेक विपरीत धारणायें फैल गयी हैं।श्रद्धा  अनेक पाखण्डी गुरु लोगों की धार्मिक भावनाओं का अनुचित लाभ उठाते हुये श्रद्धा शब्द की आड़ में विपुल धन अर्जित करते हैं। श्रद्धा शब्द का अर्थ अन्धविश्वास करके सामान्य भक्त जनों के बौद्धिक एवं मानसिक विकास की सर्वथा उपेक्षा कर दी जाती है। श्रद्धा का अर्थ यह नहीं है कि दैवी माने जाने वाली किसी घोषणा को बिना सोचे समझे वैसे ही स्वीकार कर लिया जाय।श्री शंकराचार्य के अनुसार श्रद्धा वह है जिसके द्वारा मनुष्य शास्त्र एवं आचार्य द्वारा दिये गये उपदेश से तत्त्व का यथावत् ज्ञान प्राप्त कर सकता है।तत्पर  आत्मविकास के किसी भी मार्ग पर अग्रसर साधक के लिये आवश्यक है कि वह उस मार्ग की ओर पूर्ण ध्यान दे तथा मन में ईश्वर का स्मरण रखे। शास्त्रों का केवल बौद्धिक अध्ययन हमें अन्तकरण शुद्धि प्रदान नहीं कर सकता। मन और बुद्धि को संगठित करके उपनिषदों में उपदिष्ट जीवन जीना चाहिये।जितेन्द्रिय  आत्मसंयम के बिना श्रद्धा और ज्ञान में दृढ़ता आनी कठिन है। इन्द्रियां ही हमें विषयों की ओर आकर्षित करके खींच ले जाती हैं। एक बार वैषयिक उपभोगों में आसक्त हो जाने पर जीवन के उच्च मूल्यों को बनाये रखना संभव नहीं होता। दिव्य मार्ग पर चलने का अर्थ है विषयोपभोग की नालियों से बाहर निकल जाना। ये दोनों प्रकार के जीवन परस्पर विरोधी हैं। एक की उपस्थिति में दूसरे का अभाव होता है। जहाँ हृदय में शान्ति के प्रकाश का उदय हुआ वहाँ वैषयिक और पाशविक प्रवृत्तियों से उत्पन्न क्षोभ का अन्धकार नष्ट हो जाता है अस्तु साधक के लिये आत्मसंयम का जीवन अनिवार्य हो जाता है।विषय सुख का त्याग कर स्वयं में तथा शास्त्रों में विश्वास रखते हुए दिव्य लक्ष्य को ही प्राप्त करने का प्रयत्न क्यों करना चाहिये  साधना की प्रारम्भिक अवस्था में साधक बुद्धि के स्तर पर ही रहता है और बुद्धि का कार्य प्रत्येक वस्तु के कारण की खोज करना है। स्वाभाविक है कि विचारशील पुरुष के मन में प्रश्न उठेगा कि आखिर विषय सुख के त्य़ाग का फल क्या होगा  दूसरी पंक्ति में इसका उत्तर दिया गया है।उपनिषदों के मन्त्रद्रष्टा ऋषियों का यह निश्चयात्मक आश्वासन है कि श्रद्धावान तत्पर और जितेन्द्रिय पुरुष आत्मज्ञान को प्राप्त करता है। यहां भगवान् कहते हैं कि इस ज्ञान का फल है परम शान्ति। पूर्व श्लोक के समान यहां भी इस शान्ति को प्राप्त करने का निश्चित समय नहीं बताया गया क्योंकि वह साधक के प्रयत्न पर निर्भर करता है। परन्तु यह निश्चित है कि ज्ञान को प्राप्त कर शीघ्र ही वह परम शान्ति को प्राप्त होता है।परमशान्ति  परम का अर्थ है अनन्त। अत परम शान्ति वह है जो कभी क्षीण नहीं होती। आज के युग में जहां शान्ति के नाम पर युद्ध होते रहते हैं वहां इस श्लोक में निर्दिष्ट शान्ति को भी कोई व्यक्ति शंका की दृष्टि से देखे तो कोई आश्चर्य नहीं। समयसमय पर शान्ति वार्ता करने वाले राजनीतिज्ञों की यह शान्ति नहीं है')
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
490,13,8,अमानित्वमदम्भित्वमहिंसा क्षान्तिरार्जवम्।आचार्...,क्षेत्र और क्षेत्रज्ञ का ज्ञान प्राप्त करना के...
491,13,9,इन्द्रियार्थेषु वैराग्यमनहङ्कार एव च।जन्ममृत्य...,क्षेत्र और क्षेत्रज्ञ का ज्ञान प्राप्त करना के...
492,13,10,असक्ितरनभिष्वङ्गः पुत्रदारगृहादिषु।नित्यं च सम...,क्षेत्र और क्षेत्रज्ञ का ज्ञान प्राप्त करना के...
493,13,11,मयि चानन्ययोगेन भक्ितरव्यभिचारिणी।विविक्तदेशसे...,क्षेत्र और क्षेत्रज्ञ का ज्ञान प्राप्त करना के...
494,13,12,अध्यात्मज्ञाननित्यत्वं तत्त्वज्ञानार्थदर्शनम्।...,क्षेत्र और क्षेत्रज्ञ का ज्ञान प्राप्त करना के...
82,2,41,व्यवसायात्मिका बुद्धिरेकेह कुरुनन्दन।\n\nबहुशा...,आसक्ति मानसिक क्रिया है। इसका अभिप्राय यह है क...
528,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,श्रीकृष्ण पुनः दोहराते हैं कि किस प्रकार से त...
529,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,श्रीकृष्ण पुनः दोहराते हैं कि किस प्रकार से त...
530,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,श्रीकृष्ण पुनः दोहराते हैं कि किस प्रकार से त...
129,3,16,एवं प्रवर्तितं चक्रं नानुवर्तयतीह यः।\n\nअघायु...,चक्र या चरण से तात्पर्य घटनाओं की क्रमबद्धता स...
